In [1]:
%load_ext autoreload
%autoreload 

In [2]:
import lib
import torch, numpy as np, pandas as pd, pytorch_lightning as pl, matplotlib.pyplot as plt
from torch import nn
from torchvision.transforms.functional import resize

/home/26a_kir@lab.graphicon.ru/miniconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
opts = lib.get_default_opts()
opts['visualize_saliency'] = False

In [4]:
loaders = lib.get_loaders(opts)

In [5]:
batch = next(iter(loaders['valid']))

In [6]:
batch['image'].shape, batch['saliency'].shape

(torch.Size([16, 3, 384, 512]), torch.Size([16, 1, 384, 512]))

In [7]:
device = torch.device('cpu')
model = lib.Model(opts)

In [8]:
trainer = pl.Trainer(
    accelerator='gpu',
    devices=[0]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
trainer.validate(model, loaders['test_clive'])

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Validation: 0it [00:00, ?it/s]

[Epoch   0] VALID plcc: 0.83, srocc: 0.82, loss: 239.71 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val/loss             239.7073211669922
        val/plcc            0.8289004029046462
        val/srocc           0.8195338594611115
        val_srocc           0.8195338594611115
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val/srocc': 0.8195338594611115,
  'val/plcc': 0.8289004029046462,
  'val/loss': 239.7073211669922,
  'val_srocc': 0.8195338594611115}]

In [2]:
from tqdm.notebook import tqdm
from time import sleep
for _ in range(100):
    sleep(0.5)

KeyboardInterrupt: 